<a href="https://colab.research.google.com/github/alysondv/Fundamentos-ETL/blob/main/Aulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neste estudo será utilizado o dataset de ocorrências aeronáuticas na aviação civil Brasileira disponibilizado pela iniciativa [Opendata AIG Brazil](https://dados.gov.br/dataset/ocorrencias-aeronauticas-da-aviacao-civil-brasileira).

Imports necessários

In [8]:
import pandas as pd

### Extração

In [5]:
!wget "http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv"

--2021-11-09 00:41:12--  http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv
Resolving sistema.cenipa.aer.mil.br (sistema.cenipa.aer.mil.br)... 189.9.13.91
Connecting to sistema.cenipa.aer.mil.br (sistema.cenipa.aer.mil.br)|189.9.13.91|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 916054 (895K) [text/csv]
Saving to: ‘ocorrencia.csv’

ocorrencia.csv      100%[===================>] 894.58K   320KB/s    in 2.8s    

2021-11-09 00:41:17 (320 KB/s) - ‘ocorrencia.csv’ saved [916054/916054]



 Ler o arquivo que contem os dados que queremos transformar em informação

In [7]:
ds = pd.read_csv('ocorrencia.csv', sep=";", parse_dates=['ocorrencia_dia'])
ds.dtypes

codigo_ocorrencia                          int64
codigo_ocorrencia1                         int64
codigo_ocorrencia2                         int64
codigo_ocorrencia3                         int64
codigo_ocorrencia4                         int64
ocorrencia_classificacao                  object
ocorrencia_latitude                       object
ocorrencia_longitude                      object
ocorrencia_cidade                         object
ocorrencia_uf                             object
ocorrencia_pais                           object
ocorrencia_aerodromo                      object
ocorrencia_dia                    datetime64[ns]
ocorrencia_hora                           object
investigacao_aeronave_liberada            object
investigacao_status                       object
divulgacao_relatorio_numero               object
divulgacao_relatorio_publicado            object
divulgacao_dia_publicacao                 object
total_recomendacoes                        int64
total_aeronaves_envo

Para facilitar o processo de entendimento das etapas que serão realizadas durante o estudo, algumas features serão excluídas, que são:
* codigo_ocorrencia1,
* codigo_ocorrencia3,
* codigo_ocorrencia4,
* ocorrencia_latitude,
* ocorrencia_longitude,
* ocorrencia_pais,
* investigacao_aeronave_liberada,
* investigacao_status,
* divulgacao_relatorio_numero,
* divulgacao_relatorio_publicado,
* total_aeronaves_envolvidas e 
* ocorrencia_saida_pista.

In [12]:
ds = ds.drop(['codigo_ocorrencia1','codigo_ocorrencia3','codigo_ocorrencia4','ocorrencia_latitude',
         'ocorrencia_longitude','ocorrencia_pais','investigacao_aeronave_liberada',
         'investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado',
         'total_aeronaves_envolvidas','ocorrencia_saida_pista'], axis=1)

In [13]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6114 entries, 0 to 6113
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   codigo_ocorrencia          6114 non-null   int64         
 1   codigo_ocorrencia2         6114 non-null   int64         
 2   ocorrencia_classificacao   6114 non-null   object        
 3   ocorrencia_cidade          6114 non-null   object        
 4   ocorrencia_uf              6114 non-null   object        
 5   ocorrencia_aerodromo       6114 non-null   object        
 6   ocorrencia_dia             6114 non-null   datetime64[ns]
 7   ocorrencia_hora            6113 non-null   object        
 8   divulgacao_dia_publicacao  1577 non-null   object        
 9   total_recomendacoes        6114 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 477.8+ KB
